## Предсказание траекторий автомобилей

In [ ]:
from IPython.display import Video

In [ ]:
Video("motion-prediction-video.mp4", width=400, height=400)

## Соревнование https://research.yandex.com/shifts/vehicle-motion-prediction

Трек по предсказанию траекторий движения автомобилей. Соревнование в целом посвящено исследованию подходов к оценке неопределенности и устойчивости моделей к сдвигам во входных данных.

![Ансамбль моделей](uncertainty.jpeg)

## Что представляют из себя данные

Единицей данных в датасете является сцена. Сцена содержит в себе информацию о динамических объектах (автомобилях и пешеходах), векторную карту дорожного графа, состояния светофоров, теги для сцены и отдельных агентов.

Данные в сцене разбиты на две зоны: прошлое и будущее. Для каждого автомобиля нам известны следующие параметры:
- уникальный идентификатор
- положение в глобальной системе координат
- размеры
- скорость
- ускорение

Аналогичные данные известны для самого беспилотника. Для пешехедов данные ограничиваются id, положением, размером и скоростью.

Дорожный граф содержит в себе информацию о полосах, полигон дороги, полигоны пешеходных переходов.

Ссылка на тест сет, с которым поработаем на семинаре: https://disk.yandex.ru/d/M_4ED0r19OnSrg

## API для работы с датасетом

Рекомендую установить в virtualenv, чтобы не ставить лишние пакеты в хостовую систему.
```
git clone git@github.com:yandex-research/shifts.git
cd shifts/sdc
pip install .
```

Внимание!!! При такой установке может возникнуть проблема с тем, что старые(<=3.21) и новые версии протобуфа(>=4) несовместимы. Чтобы заработало, пришлось явно прописать некоторые версии при установке, файлик с новыми requirement.txt лежит рядом с ноутбуком. Версии из файлика также надо прописать в setup.py

## Посмотрим на данные

Исходные сырые данные хранятся в формате protubuf (https://developers.google.com/protocol-buffers), он позволяет удобно хранить и рабоать со структурированными объектами.

In [ ]:
from ysdc_dataset_api.utils import read_scene_from_file

In [ ]:
scene = read_scene_from_file('test/011/011001.pb')

In [ ]:
print(f'Тип объекта: {type(scene)}')
print(f'Поля объекта: {[field for field in dir(scene) if not field.startswith("_") and field.islower()]}')

Каждая сцена содержит в себе данные о прошлом и будущем на 5 секунд (всего 10 секунд). Это время разбито на 50 дискретных таймстемпов с частотой  5Hz.

In [ ]:
print(f'Горизонт прошлого: {len(scene.past_vehicle_tracks)}, горизонт будущего: {len(scene.future_vehicle_tracks)}')

Индекс 0 в past_tracks соответсвтует -5 секундам в истории, индекс 24 -- нулевая секунда, момент предсказния.

In [ ]:
print(f'Количество машин в момент предсказания: {len(scene.past_vehicle_tracks[-1].tracks)}')
print('Информация об одной из машин в момент предсказания:')
print(scene.past_vehicle_tracks[-1].tracks[1])

В сцене есть поле `prediction_requests` содержащее id автомобилей, для которых необходимо сделать предсказание. Помимо этого реквесты помечены тегами, описывающими характер движения автомобиля. Полный список тегов можно найти в файле `tags.proto` [ссылка](https://github.com/yandex-research/shifts/blob/main/sdc/ysdc_dataset_api/proto/tags.proto)

In [ ]:
for r in scene.prediction_requests:
    if r.track_id == scene.past_vehicle_tracks[-1].tracks[1].track_id:
        print(r)

Тэги есть так же и у сцен. Они описывают различные срезы данных, по которым проходило разбиение в соревновании.

In [ ]:
print(scene.scene_tags)

### Про классы из библиотеки, которые мы будем использовать

#### Для получения фичей из сцены есть 2 класса:
- `FeatureRenderer(renderer_config)` -- объект, который описывает генерацию фичей мапы по сцене
- `FeatureVectorizer(vectorizer_config)` -- объект, который описывает генерацию векторных фичей из истории агентов

Примеры конфигов для генерации фичей лежат рядом с ноутбуком

#### Для работы со сценами как с датасетом есть класс 
```
MotionPredictionDataset(
    dataset_path: str,
    prerendered_dataset_path: str,
    feature_producers: List[Union[FeatureRenderer, FeatureVectorizer]],
    transform_ground_truth_to_agent_frame: bool
)
```
`dataset_path` -- путь до датасета \
`prerendered_dataset_path` -- путь до датасета c пререндеренными картинками \
`feature_producers` -- cписок векторайзеров и рендереров для генерации фичей. При добавлении пререндеренных фичей их рендерер указывать не надо \
`transform_ground_truth_to_agent_frame` -- переводить ли сцену в систему координат агента

В нашем случае пререндеренные фичи лежат там же, где и объекты. `<scene>.pb` -- сцена, `<scene>.npy` -- пререндеренные фичи для сцены.

Давайте нарисуем уже сцену и посмотрим, как оно выглядит не в скучных числах.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import yaml
from matplotlib import collections as mc

from ysdc_dataset_api.dataset import MotionPredictionDataset
from ysdc_dataset_api.features import FeatureRenderer
from ysdc_dataset_api.utils import transform_2d_points

In [ ]:
with open('renderer_config.yaml') as f:
    renderer_config = yaml.safe_load(f)

In [ ]:
renderer = FeatureRenderer(renderer_config)

In [ ]:
dataset_path = 'test/'

In [ ]:
dataset = MotionPredictionDataset(
    dataset_path=dataset_path,
    feature_producers=[renderer],
    transform_ground_truth_to_agent_frame=True)

In [ ]:
print(f'Количество сцен в датасете: {dataset.num_scenes}')

In [ ]:
dataset_iter = iter(dataset)

# Проитерируемся по датасету в поисках машины, проехавшей более 10 метров по обеим из координат.
while True:
    data_item = next(dataset_iter)
    if data_item['ground_truth_trajectory'][-1, 0] > 10.0 and data_item['ground_truth_trajectory'][-1, 1] > 10.0:
        break

In [ ]:
print(f'Датасет для каждого объекта возвращает набор следующих полей: {[k for k in data_item.keys()]}')

In [ ]:
print(f'Размеры фичемап, которые нам отдал рендерер: {data_item["feature_maps"].shape}')

In [ ]:
# Plot vehicles occupancy, pedestrian occupancy, lane occupancy and road polygon
plt.figure(figsize=(10, 10))
plt.imshow(data_item['feature_maps'][0], origin='lower', cmap='binary', alpha=0.7)
plt.imshow(data_item['feature_maps'][6], origin='lower', cmap='binary', alpha=0.5)
plt.imshow(data_item['feature_maps'][13], origin='lower', cmap='binary', alpha=0.2)
plt.imshow(data_item['feature_maps'][16], origin='lower', cmap='binary', alpha=0.1)

# Переведем ground truth траекторию агента в систему координат фичемапы
transformed_gt = transform_2d_points(data_item['ground_truth_trajectory'], renderer.to_feature_map_tf)
transformed_gt = np.round(transformed_gt - 0.5).astype(np.int32)

ax = plt.gca()
ax.add_collection(mc.LineCollection([transformed_gt], color='green'))

## Метрики

Одними из общепринятых метрик для оценки качества предсказания траекторий являются Average Displacement Error и Final Displacement Error, а так же их модификации minADE@k, minFDE@k:
- ADE - среднее по таймстемпам L2 отклонеие предсказания от ground truth
- FDE - L2 отклонение последней предсказанной точки траектории от ground truth
- minADE@k -- минимальное значение ADE по k наиболее вероятных предсказанных гипотез
- minFDE@k -- минимальное значение FDE по k наиболее вероятных предсказанных гипотез

In [ ]:
def ade(y_true, y_pred):
    """
    Insert your code for ADE computation below.
    
    Args:
        y_true (np.ndarray): shape (batch, n_timestamps, 2)
        y_pred (np.ndarray): shape (batch, n_timestamps, 2)

    Returns:
        np.ndarray: shape (batch, 1)
    """

In [ ]:
def fde(y_true, y_pred):
    """
    Insert your code for FDE computation below.
    
    Args:
        y_true (np.ndarray): shape (batch, n_timestamps, 2)
        y_pred (np.ndarray): shape (batch, n_timestamps, 2)

    Returns:
        np.ndarray: shape (batch, 1)
    """

In [ ]:
def min_ade(y_true, y_pred):
    """
    Insert your code for minADE computation below.
    
    Args:
        y_true (np.ndarray): shape (batch, n_timestamps, 2)
        y_pred (np.ndarray): shape (batch, n_modes, n_timestamps, 2)

    Returns:
        np.ndarray: shape (batch, 1)
    """

In [ ]:
def min_fde(y_true, y_pred):
    """
    Insert your code for minFDE computation below.
    
    Args:
        y_true (np.ndarray): shape (batch, n_timestamps, 2)
        y_pred (np.ndarray): shape (batch, n_modes, n_timestamps, 2)

    Returns:
        np.ndarray: shape (batch, 1)
    """

## Модель с константным предсказанием

In [ ]:
import torch
import tqdm

from ysdc_dataset_api.features import FeatureVectorizer

In [ ]:
class BaselineModel(torch.nn.Module):
    def __init__(self, gt_time_grid):
        super().__init__()
        self._gt_time_grid = torch.tensor(gt_time_grid)
    
    def forward(self, velocity):
        states = torch.einsum('bc,t->btc', velocity, self._gt_time_grid)
        return states

In [ ]:
model = BaselineModel(np.linspace(0.2, 5, 25))

In [ ]:
with open('vectorizer_config.yaml') as f:
    vectorizer_config = yaml.safe_load(f)
vectorizer = FeatureVectorizer(vectorizer_config)

dataset = MotionPredictionDataset(
    dataset_path='test/',
    prerendered_dataset_path='test/',
    feature_producers=[vectorizer],
    transform_ground_truth_to_agent_frame=True,
)

In [ ]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=32,
    num_workers=8,
)

In [ ]:
ades = []
fdes = []
for batch in tqdm.tqdm(dataloader):
    # Возьмём скорость по (x, y) в последний известный момент времени
    predictions = model(batch['vector_features'][:, -1, 2:4])
    ades.append(ade(batch['ground_truth_trajectory'], predictions))
    fdes.append(fde(batch['ground_truth_trajectory'], predictions))

In [ ]:
np.mean(np.concatenate(ades)), np.mean(np.concatenate(fdes))

## Визуализация предсказаний

In [ ]:
# Тк в датасете выше мы использовали готовые пре-рендеренные картинки (см prerendered_dataset_path),
# сделаем себе соответсвующий рендерер, чтобы извлечь из него трансформ для визуализации.
with open('prerendered_images_config.yaml') as f:
    renderer_config = yaml.safe_load(f)
renderer = FeatureRenderer(renderer_config)


# Plot vehicles occupancy, pedestrian occupancy, lane occupancy and road polygon
i = 1
plt.figure(figsize=(10, 10))
plt.imshow(batch['prerendered_feature_map'][i][0], origin='lower', cmap='binary', alpha=0.7)
plt.imshow(batch['prerendered_feature_map'][i][6], origin='lower', cmap='binary', alpha=0.5)
plt.imshow(batch['prerendered_feature_map'][i][13], origin='lower', cmap='binary', alpha=0.2)
plt.imshow(batch['prerendered_feature_map'][i][16], origin='lower', cmap='binary', alpha=0.1)

# Переведем ground truth траекторию агента в систему координат фичемапы
transformed_gt = transform_2d_points(batch['ground_truth_trajectory'][i].numpy(), renderer.to_feature_map_tf)
transformed_gt = np.round(transformed_gt - 0.5).astype(np.int32)

ax = plt.gca()
ax.add_collection(mc.LineCollection([transformed_gt], color='green'))

prediction = predictions[i].numpy().astype(np.float32)
transformed_prediction = transform_2d_points(prediction, renderer.to_feature_map_tf)
ax.add_collection(mc.LineCollection([transformed_prediction], color='red'))

## Мультимодальность

#### Общий подход
Теперь мы предсказываем распределение на траекториях
$$
trajectory \sim \sum\limits_{i=1}^{M} p_i q(mode_i)
$$
где $p_i$ -- вероятность $i$-ой моды,  $q(mode_i)$ -- некоторое распределение значений $i$-ой моды

#### Имплементация
- В одномодальном случае мы предсказывали траекторию из некоторого промежуточного состояния(эмебддинга) некоторой головой.
- В многомодальном случае мы из того же самого эмбеддинга предсказываем несколько голов, каждая голова отвечает за свою моду. Дополнительно из эмбеддинга предсказываем логиты, из которых получим вероятности реализации мод.
#### Лосс
Обозначи за $gt$ -- ground truth траекторию, $mode_i$ -- предсказанное значение $i$-ой моды \
Пусть $\widehat{i}$ -- индекс моды с ближайшим к GT предсказанием(т. е.  $\widehat{i} = \arg\min\limits_i ADE(mode_i, gt)$) 
- Взвешенный ADE
$$loss = \sum\limits_{i=1}^{M} p_i ADE(mode_i, gt)$$
может сходиться к предсказанию с нулевыми вероятностями всех мод кроме одной
- ADE для ближайшей траектории + классификация ближайшей траектории \
$$loss = -\log{p_{\widehat{i}}} + \alpha ADE(mode_{\widehat{i}}, gt)$$
- GMM(Gaussian mixture model) \
Модель предсказывает для моды с индексом $i$ среднее и стандартное отклонение ($\mu_{i}$, $\sigma_{i}$)
$$loss = -\log{p_{\widehat{i}}} - \alpha \log{\mathcal{N}(gt | \mu_{\widehat{i}}, \sigma_{\widehat{i}})}$$

## Практические аспекты

### Типичная схема сети для multitask learning

![Схема моделей](schema.png)

#### Виды голов
- Траектория
- Интент (стационарность, уступание, движение задним ходом и т д)
- Обусловленная траектория:
  - На интент. Данная голова сожержит траектории для всех значений интента. Каждая траектория обучается на подмножестве данных, где реализовалось соответствующее значение интента
  - На траекторию. Подмешиваем к эмбеддингу предсказываемого агента эмбеддинг траектории(это может быть траектория ровера, lane path и т д)

Чем больше адекватных голов, тем лучше. Используем multitask learning. 

#### Двустадийность 
В продакшне мы ограничены требованиями к задержкам системы и должны выдавать траектории за определенное время. Для того, чтобы была возможность увеличить сетку без увеличения задержек, можно использовать следующий подход:
- Используем выход энкодера графа дорог и другой медленно меняющейся информации с предыдущего тика.
- Для быстро изменяющихся фичей агентов считаем данные с текущего тика
- Опционально можно добавить смещение нашей позиции за тик, чтобы помочь сетке понимать, как изменился граф полос с предыдущего тика

#### Данные
- Проезды водителей "на руках"
- Автоматические проезды:
  -  active learning, семплируем моменты, где ошиблась текущая модель
      - Ни одна мода не лежит в окрестности наблюдаемой траектории агента
      - Неправильно определен интент
  - интересные моменты (с сдвигами назад и вперед)
      - перестроение
      - проехали пешеходный переход
      - объезд препятствия
      - замедления
      - ускорения
      - и т д 

## Задание

В качестве задания предлагается обучить нейросетку для предсказания траекторий и побить наш бейзлайн. В качестве решения ожидается тетрадка с кодом (которым можно воспроизвести полученные результаты). Тетрадка также должна содержать подробный отчёт о рассмотренных подходах, сравнение подходов на тесте между собой, а также с бейзлайном.
Ссылки на данные:
- [трейн](https://disk.yandex.ru/d/tuTwRSLL-KFqjg)
- [валидация](https://disk.yandex.ru/d/3Lu6_6BgwkXlgw)
- [тест](https://disk.yandex.ru/d/M_4ED0r19OnSrg)

Нужно провести следующие эксперименты:
1. Обучить любую нейросетевую модель(без добавления картинок/векторного представления графа), которая сошлась и предсказывает визуально разумные траектории **(4 балла)**
2. Обучить сетку с добавлением картинок/векторного представления графа, правда если от добавления сигнала стало хуже, ставим 0 баллов **(4 балла)**
3. Обучить мультимодальное предсказание, если вы покажете хотя бы один пример, где гипотезы заметно отличаются и при этом у обоих гипотез не околонулевые вероятности. Кроме того, minADE этой сетки должен быть лучше, чем сетки из пункта 2 **(2 балла)**

В каждом пункте нужны картинки, без картинок будем снижать баллы
Если в каком-то пункте предсказание на тесте получилось хуже кинематического предсказания (бейзлайна с семинара), то оценку за пункт делим на 2.

In [ ]:
# Для использования картиночных фичей можно использовать уже нарендеренные картинки (они есть в датасете).
# Рендерить с нуля может быть довольно долго.
# Пример использования готовых картинок в датасете:

# dataset = MotionPredictionDataset(
#     dataset_path='test/',
#     prerendered_dataset_path='test/',  # Тут мы указываем путь, где искать готовые картинки
#     feature_producers=[vectorizer],
#     transform_ground_truth_to_agent_frame=True,
# )